In [24]:
import json
import numpy as np
from helpers.helper import get_cath

In [6]:
cath = get_cath()

In [21]:
def is_disc(chain):
	domains = cath[chain[:-2]][chain[-1]]
	for _, bounds in domains.items():
		if ',' in bounds:
			return True
	return False

In [19]:
chain = '10mh:A'
domains = cath[chain[:-2]][chain[-1]]
for _, bounds in domains.items():
    if ',' in bounds:
        print(True, bounds)

True 1-186,285-327


In [7]:
with open('../data/cath/iid/chains_to_seq_iid.json', 'r') as jf:
	chains_to_seq = json.load(jf)

In [22]:
disc, non_dics = 0, 0
for chain in chains_to_seq.keys():
    if is_disc(chain):
        disc += 1
    else:
        non_dics += 1
        
print("Non-Discontinuous count:", non_dics)
print("Discontinuous count:", disc)

Non-Discontinuous count: 6545
Discontinuous count: 1952


In [28]:
def boundaries_to_domains(binary_vector):
    binary_vector = np.copy(binary_vector)
    domain = 1
    for i in range(len(binary_vector)):
        if binary_vector[i] == 1:
            # this is the start of a new domain
            domain += 1
        binary_vector[i] = domain
    return binary_vector

In [29]:
boundaries_to_domains([0,0,0,0,1,0,0,0,0,1,0])

array([1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3])

In [93]:
def ndo(y_pred, y_true):
    y_pred = boundaries_to_domains(y_pred)
    y_true = boundaries_to_domains(y_true)
    table = np.zeros((y_true[-1]+1, y_pred[-1]+1))
    
    for d_pred, d_true in zip(y_pred, y_true):
        table[d_true-1, d_pred-1] += 1

    print(table.shape)
    print(table)

    # columns
    for i in range(table.shape[1]-1):
        table[-1, i] = 2 * max(table[:, i]) - sum(table[:, i])

    # rows
    for i in range(table.shape[0]-1):
        table[i, -1] = 2 * max(table[i, :]) - sum(table[i, :])

    table[-1, -1] = (sum(table[:, -1]) + sum(table[-1, :])) / 2

    score = table[-1, -1] / len(y_true)

    print(table)

    return score


In [94]:
y_true = [0] * 301
y_pred = [0] * 301

y_true[85] = 1
y_pred[85] = 1

y_pred[85+134] = 1


ndo(y_pred, y_true)

(3, 4)
[[ 85.   0.   0.   0.]
 [  0. 134.  82.   0.]
 [  0.   0.   0.   0.]]
[[ 85.   0.   0.  85.]
 [  0. 134.  82.  52.]
 [ 85. 134.  82. 219.]]


0.7275747508305648